In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import timm
from tqdm import tqdm
import torch.nn.functional as F
from attack_methods import fgsm_attack, pgd_attack


import numpy as np

def patch_wise_drop(image, num_patches, patch_size=16):
    # Assuming image is a Tensor of shape (C, H, W)
    _, rows, cols = image.shape
    patched_image = image.clone()
    
    # Calculate the number of possible patches in both dimensions
    num_possible_patches_x = cols // patch_size
    num_possible_patches_y = rows // patch_size
    
    # Create a grid of all possible patches
    all_patches = [(x * patch_size, y * patch_size) for x in range(num_possible_patches_x) for y in range(num_possible_patches_y)]
    
    # Randomly choose num_patches patches from the list of all possible patches
    chosen_patches = torch.randperm(len(all_patches))[:num_patches]
    
    for index in chosen_patches:
        x, y = all_patches[index]
        # Since we're dealing with a Tensor, we need to fill the patch with zeros
        patched_image[:, y:y + patch_size, x:x + patch_size] = 0  # Assuming that 0 is the value you want to use for masking

    return patched_image

def pixel_wise_drop(image, num_patches, patch_size=16):
    c, h, w = image.shape
    total_pixels = h * w
    patch_area = patch_size * patch_size
    total_patch_area = patch_area * num_patches
    drop_ratio = total_patch_area / total_pixels

    # Create a flattened array of all pixel indices
    pixel_indices = np.arange(h * w)
    # Randomly choose a subset of pixel indices to drop
    drop_indices = np.random.choice(pixel_indices, int(h * w * drop_ratio), replace=False)

    # Create a mask of ones
    mask = np.ones((h * w), dtype=np.float32)
    # Set chosen pixel indices to zero in the mask to 'drop' them
    mask[drop_indices] = 0
    mask = mask.reshape((h, w))

    # Apply the mask to the image
    image_np = image.cpu().numpy().copy()
    for i in range(c):
        image_np[i] *= mask

    return torch.from_numpy(image_np).float()


def shuffle_patches(image, grid_size):
    """
    Shuffles patches within an image tensor.

    Parameters:
    image (Tensor): A PyTorch tensor representing the image.
    grid_size (int): The number of patches along one dimension. For example, grid_size=2 creates a 2x2 grid.

    Returns:
    Tensor: A PyTorch tensor of the shuffled image.
    """
    C, H, W = image.shape
    patch_H, patch_W = H // grid_size, W // grid_size

    # Check if image dimensions are divisible by grid_size
    if H % grid_size != 0 or W % grid_size != 0:
        raise ValueError("Image dimensions must be divisible by grid_size.")

    # Reshape the image to have dimensions (C, grid_size, patch_H, grid_size, patch_W)
    patches = image.view(C, grid_size, patch_H, grid_size, patch_W)
    
    # Transpose to have dimensions (grid_size, grid_size, C, patch_H, patch_W)
    patches = patches.permute(1, 3, 0, 2, 4)
    
    # Reshape to (grid_size*grid_size, C, patch_H, patch_W) to shuffle
    patches = patches.contiguous().view(grid_size * grid_size, C, patch_H, patch_W)
    
    # Shuffle patches
    shuffle_indices = torch.randperm(patches.shape[0])
    patches = patches[shuffle_indices]
    
    # Reshape back to grid
    patches = patches.view(grid_size, grid_size, C, patch_H, patch_W)
    
    # Transpose to (C, grid_size, patch_H, grid_size, patch_W)
    patches = patches.permute(2, 0, 3, 1, 4)
    
    # Reshape back to the original image shape using reshape instead of view
    shuffled_image = patches.reshape(C, H, W)
    
    return shuffled_image


def add_gaussian_noise(image, std=0.1, mean=0):
    """
    Adds Gaussian noise to an image tensor.

    Parameters:
    image (Tensor): A PyTorch tensor representing the image with shape (C, H, W).
    mean (float): The mean of the Gaussian noise distribution.
    std (float): The standard deviation of the Gaussian noise distribution.

    Returns:
    Tensor: A PyTorch tensor of the image with added Gaussian noise.
    """
    noise = torch.randn(image.size()) * std + mean
    noisy_image = image + noise
    # Clamp the noisy image to be within the valid range [0, 1]
    noisy_image = torch.clamp(noisy_image, 0, 1)
    return noisy_image



class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = []
        self.img_dir = img_dir
        self.transform = transform
        self.original_size = {}  # 存储原始图像尺寸
        with open(annotations_file, 'r') as f:
            for line in f:
                path, label = line.strip().split()
                self.img_labels.append((path, int(label)))

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels[idx][0])
        image = Image.open(img_path).convert('RGB')
        self.original_size[idx] = image.size  # 存储原始图像尺寸
        if self.transform:
            image = self.transform(image)
        return image, self.img_labels[idx][1], idx  # 返回原始尺寸的索引


def generate_adversarial_samples(source_root_dir, data_path_names,
                                 txt_files, adv_file_name, transform, 
                                 attck_method, attack_settings):
    for i_data_name, dataset_name in enumerate(data_path_names):
        print(dataset_name)
        
        for txt_file in txt_files:
            print(txt_file)
            annotations_file = os.path.join(source_root_dir, dataset_name, txt_file)
            img_dir = os.path.join(source_root_dir, dataset_name)
            save_dir = os.path.join(source_root_dir, dataset_name, 'images', adv_file_name)
            os.makedirs(save_dir, exist_ok=True)

            dataset = CustomImageDataset(
                annotations_file=annotations_file,
                img_dir=img_dir,
                transform=transform
            )
            dataloader = DataLoader(dataset, batch_size=16, shuffle=False)
            for images, labels, idxs in tqdm(dataloader):

                if attck_method == 'patch_drop':
                    # Call the patch_wise_drop function for each image
                    adv_images = torch.stack([patch_wise_drop(image, attack_settings['num_patches'], attack_settings['patch_size']) for image in images])
                elif attck_method == 'pixel_drop':
                    # Call the pixel_wise_drop function for each image
                    adv_images = torch.stack([pixel_wise_drop(image, attack_settings['num_patches'], attack_settings['patch_size']) for image in images])
                elif attck_method == 'patch_shuffle':
                    adv_images = torch.stack([shuffle_patches(image, attack_settings['grid_size']) for image in images])
                elif attck_method == 'gaussian_noise':
                    adv_images = torch.stack([add_gaussian_noise(image, attack_settings['std']) for image in images])
                    
                adv_images = adv_images.cpu()

                for img, idx in zip(adv_images, idxs):
                    original_size = dataset.original_size[idx.item()]
                    img_pil = transforms.ToPILImage()(img).resize(original_size)

                    img_path = dataset.img_labels[idx.item()][0]
                    save_path = os.path.join(save_dir, os.path.basename(img_path))
                    img_pil.save(save_path)

    print("完成对抗样本的生成和保存。")

In [ ]:
import time

start_time = time.time()  # 获取开始时间
# 放置你的代码

source_root_dir = '/media/ruanjiacheng/新加卷/ecodes/Prompt/data/vtab-1k'
data_path_names=("caltech101", "cifar", 
                 "clevr_count", "clevr_dist", 
                 "diabetic_retinopathy", "dmlab", "dsprites_loc", "dsprites_ori", 
                 "dtd", "eurosat", "oxford_flowers102", "kitti", "patch_camelyon", 
                 "oxford_iiit_pet", "resisc45", "smallnorb_azi", "smallnorb_ele", "sun397", "svhn"
                )
data_weights_names=("caltech101", "cifar100",
                    "clevr_count", "clevr_dist", 
                 "diabetic_retinopathy", "dmlab", "dsprites_loc", "dsprites_ori", 
                 "dtd", "eurosat", "flowers102", "kitti", "patch_camelyon", 
                 "pets", "resisc45", "smallnorb_azi", "smallnorb_ele", "sun397", "svhn"
                )
dataset_classes=(102, 100,
                 8, 6, 5, 6, 16, 16, 47, 10, 102, 4, 2, 37, 45, 18, 9, 397, 10
                )
# txt_files = ['test.txt', 'train800.txt', 'train800val200.txt', 'val200.txt']
txt_files = ['test_adv_500.txt']

model_name = 'vit_base_patch16_224_in21k'
tuning_mode = ['linear_prob']
tuning_coeff=0
weight_root_dir='/media/ruanjiacheng/新加卷/ecodes/Prompt/CV/GIST_ALL/outputs_adv/[linear_probe]_0'

# 图像转换，无变化
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


attck_method='gaussian_noise'
# attack_settings = {
#     'num_patches': 140,
#     'patch_size': 16
# }
# attack_settings = {
#     'grid_size': 14
# }
attack_settings = {
    'std': 1.0
}

# adv_file_name = f'{attck_method}_num-{attack_settings["num_patches"]}_size-{attack_settings["patch_size"]}'
adv_file_name = f'{attck_method}std-{attack_settings["std"]}'


generate_adversarial_samples(source_root_dir, data_path_names,
                                 txt_files, adv_file_name, transform, 
                                 attck_method, attack_settings)

end_time = time.time()  # 获取结束时间
print(f"执行时间：{end_time - start_time} 秒")